In [1]:
#tworze słownik unigramowy [słowo] : [ilość wystąpień słowa]
unigram_dict = {}
with open('./../ruznezecy/1grams.txt', 'r', encoding='utf8') as unigrams:
    for line in unigrams:
        frequency, word = line.split()
        unigram_dict[word] = frequency


In [2]:
# nowy słownik: [słowo] : [tag]
tag_dict = {}
with open('./../ruznezecy/supertags.txt', 'r', encoding='utf8') as supertags:
    for line in supertags:
        word, tag = line.split()
        tag_dict[word] = tag

In [3]:
# Tutaj przygotowuję sobie słowniki składające się z klucz(słowo) : wartość[[następnik1,częstość występowania],...
bigrams_weight_dict = {}
with open('./../ruznezecy/poleval_2grams.txt', 'r', encoding='utf8') as bigrams:
    for line in bigrams:
        current_line = line.split()
        frequency, first, second = current_line[0], current_line[1] , current_line[2]
        if int(frequency) > 5: #biorę tylko bigramy częstsze niż 5
            if first in bigrams_weight_dict.keys():
                bigrams_weight_dict[first].append([second, frequency])
            else:
                bigrams_weight_dict[first] = [[second, frequency]]         

In [4]:
UNIGRAMS_AMOUNT = 0
BIGRAMS_AMOUNT = 0
for line in unigram_dict:
   UNIGRAMS_AMOUNT += int(unigram_dict[line])

for line in bigrams_weight_dict:
    for pair in bigrams_weight_dict[line]:
        BIGRAMS_AMOUNT += int(pair[1])

In [5]:
# [tag] : [ilość wystąpień]

tag_unigram_frequency = {}
for word in tag_dict:
    word_tag = tag_dict[word]
    if word in unigram_dict.keys():
        if word_tag in tag_unigram_frequency.keys():
            tag_unigram_frequency[word_tag] += int(unigram_dict[word])
        else:
            tag_unigram_frequency[word_tag] = int(unigram_dict[word])


In [6]:
def end_of_word(word):
    return ('^' + word)[-3:]

def find_tag(w):  
    if w in ['.', ',', ';', ':']:
        return None
    else:
        if w in tag_dict.keys():
            return tag_dict[w]
        else:
            return None
    # for tag in tag_dict:
    #     if w in tag_dict[tag]:
    #         return tag
    # for tag in tag_dict:
    #     if end_of_word(w) == end_of_word(tag_dict[tag][0]):
    #         return tag
  

In [7]:
tag_bigram_frequency = {}
with open('./../ruznezecy/poleval_2grams.txt', 'r', encoding='utf8') as bigrams:
    for line in bigrams: 
        current_line = line.split()
        frequency, first, second = current_line[0], current_line[1] , current_line[2]
        tag_first = find_tag(first)
        tag_second = find_tag(second)
        if tag_first != None and tag_second != None:
            if tag_first in tag_bigram_frequency.keys():
                if tag_second not in tag_bigram_frequency[tag_first]:
                    tag_bigram_frequency[tag_first][tag_second] = int(frequency)
                else:
                    tag_bigram_frequency[tag_first][tag_second] += int(frequency)
            else:
                tag_bigram_frequency[tag_first] = {tag_second: int(frequency)}


In [8]:
import math



def bigram_frequency(w1, w2):
    frequency = 0
    for pair in bigrams_weight_dict[w1]:
        if pair[0] == w2:
            frequency =  int(pair[1])
    return frequency

def bigram_tag_frequency(t1, t2):

    if t2 in tag_bigram_frequency[t1].keys():
        return tag_bigram_frequency[t1][t2]
    else:
        return 1
    
    

def bigram_tag_ppb(t1, t2):
    return bigram_tag_frequency(t1,t2)/BIGRAMS_AMOUNT

def bigram_ppb(w1, w2):
    return bigram_frequency(w1,w2)/BIGRAMS_AMOUNT

def unigram_ppb(w):
    if w in unigram_dict.keys():
        return int(unigram_dict[w])/UNIGRAMS_AMOUNT
    else:
        return 1

def unigram_tag_ppb(t1):
    if t1 in tag_unigram_frequency.keys():
        return int(tag_unigram_frequency[t1])/UNIGRAMS_AMOUNT
    else:
        return 1

def count_ppmi(w1, w2):
    return max(0, math.log(bigram_ppb(w1,w2) / (unigram_ppb(w1) * unigram_ppb(w2))))

def count_psm(w1, w2):
    return bigram_frequency(w1,w2) * (math.log(bigram_frequency(w1,w2)) - math.log(unigram_ppb(w1) * unigram_ppb(w2) * UNIGRAMS_AMOUNT) - 1)

def count_ppmi_with_tags(t1, t2):
    return max(0, math.log(bigram_tag_ppb(t1,t2) / (unigram_tag_ppb(t1) * unigram_tag_ppb(t2))))



In [9]:
def colocations_ppmi(w):
    result_dict = {}
    for pair in bigrams_weight_dict[w]:
        second_word = pair[0]
        if len(second_word) > 2:
            result_dict[w + " " + second_word] = count_ppmi(w, second_word)
        else:
            continue
    sorted_results = {k: v for k, v in sorted(result_dict.items(), key=lambda item: item[1])}
    res = dict(reversed(list(sorted_results.items())))
    counter = 0
    for elem in res:
        if counter > 10:
            break
        else:
            print(elem, res[elem])
            counter += 1

def colocations_psm(w):
    result_dict = {}
    for pair in bigrams_weight_dict[w]:
        second_word = pair[0]
        if len(second_word) > 2:
            result_dict[w + " " + second_word] = count_psm(w, second_word)
        else:
            continue
    sorted_results = {k: v for k, v in sorted(result_dict.items(), key=lambda item: item[1])}
    res = dict(reversed(list(sorted_results.items())))
    counter = 0
    for elem in res:
        if counter > 10:
            break
        else:
            print(elem, res[elem])
            counter += 1

def colocations_tags(w):
    result_dict = {}
    for pair in bigrams_weight_dict[w]:
        second_word = pair[0]
        if len(second_word) > 2:
            word1_tag = find_tag(w)
            word2_tag = find_tag(second_word)
            result_dict[w + " " + second_word] = count_ppmi_with_tags(word1_tag, word2_tag)
        else:
            continue
    sorted_results = {k: v for k, v in sorted(result_dict.items(), key=lambda item: item[1])}
    res = dict(reversed(list(sorted_results.items())))
    counter = 0
    for elem in res:
        if counter > 10:
            break
        else:
            print(elem, res[elem])
            counter += 1


test1 = "tata"


print("#" * 40, "TUTAJ WYNIKI PPMI:", "#" * 40)
colocations_ppmi(test1)
print("-" * 40)
colocations_ppmi("wielki")

print("#" * 40, "TUTAJ WYNIKI PSM:", "#" * 40)

colocations_psm(test1)
print("-" * 40)
colocations_psm("wielki")


print("#" * 40, "TUTAJ WYNIKI PPMI Z TAGAMI:", "#" * 40)

colocations_tags(test1)
print("-" * 40)
colocations_tags("wielki")


######################################## TUTAJ WYNIKI PPMI: ########################################
tata xenon 10.61574104980941
tata martino 9.316458065679148
tata steel 8.241243375225078
tata motors 8.01572426939775
tata kazika 6.789518034184597
tata group 5.233159130898281
tata kupił 3.9216432079310803
tata zmarł 3.7804220640919843
tata zabrał 3.7264892423918323
tata lubi 2.9950768134787213
tata pracował 2.9540832580396224
----------------------------------------
wielki penitencjariusz 11.38010682133685
wielki chingan 11.197785264542896
wielki przeorat 10.841110320604164
wielki gatsby 10.841110320604164
wielki kack 10.553428248152382
wielki szlem 10.34745250046054
wielki krzek 10.281494532668741
wielki moff 9.99381246021696
wielki szacun 9.844280726245996
wielki zderzacz 9.642414573379071
wielki mufti 9.634867367743688
######################################## TUTAJ WYNIKI PSM: ########################################
tata motors 277.4307374277651
tata steel 146.74929574389589
tata 